In [1]:
import numpy as np
import pandas as pd
import folium 

from gestion_donnees import data_pollution

In [2]:
QA=pd.read_csv("indices_QA_commune_IDF_04_2018.csv")


In [3]:
test=pd.DataFrame([[48.86,2.356,10],[48.90,2.35,40],[48.87,2.357,60],[48.88,2.359,90]],columns=['Latitude','Longitude','PM2.5'])
test.head()

,Latitude,Longitude,PM2.5
0,48.86,2.356,10
1,48.90,2.350,40
2,48.87,2.357,60
3,48.88,2.359,90


In [4]:
latitude = 48.8566
longitude = 2.3522
zoom=13
carte=folium.Map(location=[latitude,longitude],tiles="OpenStreetMap", zoom_start=zoom)

pollution = 42

#folium.Marker(location=[latitude,longitude], popup=f"Nox ={pollution} micro_g/m3", icon=folium.Icon(icon="cloud")).add_to(carte);

In [5]:
def niveau_couleur(value):
    if value <= 25 :
        return 'lightgreen'
    elif 25<value<=50 :
        return 'green'
    elif 50<value<=75 :
        return 'orange'
    else :
        return 'red'

In [13]:
for index, row  in test.iterrows():
    col=niveau_couleur(row['PM2.5'])
    folium.Marker(location=[row['Latitude'],row['Longitude']], tooltip=f"Nox = {row['PM2.5']} micro_g/m3", icon=folium.Icon(icon="cloud",color=col)).add_to(carte);
    


In [7]:
carte.save("carte_pollution.html")

In [14]:
display(carte)